In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import pandas as pd
from tqdm import tqdm
import pickle


In [2]:
def get_race(tokens, i):
    regex2 = "^[a-zA-z-' ]+[a-zA-Z ]+"
    
    regex_expressions = re.findall(regex2, tokens[i+1])
    
    # join into word
    
    race = " ".join(regex_expressions).strip()
    
    return race

def get_affiliation(tokens, i):
    regex3 = '[a-zA-z ]+'
    
    to_string = ' '.join(tokens[i:])
    regex_expressions = re.findall(regex3, to_string)
    regex_expressions = [k.strip() for k in regex_expressions]

        
    for i, exp in enumerate(regex_expressions):
        if "Former" in exp:
            break
    regex_expressions = regex_expressions[:i+1]
#     print(regex_expressions)
#     print((["Alliance" in i for i in regex_expressions]))
        
    
#     if "Horde" in regex_expressions:
    if any(["Horde" in i for i in regex_expressions]):
        affiliation = "Horde"
#     elif "Alliance" in regex_expressions:
    elif any(["Alliance" in i for i in regex_expressions]):
        affiliation = 'Alliance'
    else:
        affiliation = 'Other'
        
    return affiliation

In [3]:
def get_race_and_affiliation(text):

    regex1 = '[a-zA-Z][^A-Z]*'
    
    tokens_race = text.split("\n")
    tokens_affiliation = re.findall(regex1, text.replace("\n",""))

    race = '0'
    affiliation = 'Other'

    for i, word in enumerate(tokens_race):
        if 'Race' in word:
            race = get_race(tokens_race,i)
    for i, word in enumerate(tokens_affiliation):
        if 'Affiliation' in word:
            affiliation = get_affiliation(tokens_affiliation,i)
    
    return race, affiliation


In [4]:
with open('df.pickle', 'rb') as handle:
    df = pickle.load(handle)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,Other
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2229,the_Jade_Warlord,Mogu,Other
2230,the_Jailer,Eternal One,Other
2231,the_Monkey_King,Hozen,Other
2232,the_Stonewright,Venthyr,Other


In [5]:
# url = "https://wowpedia.fandom.com/wiki/Illidan_Stormrage"
# url = "https://wowpedia.fandom.com/wiki/Thrall"
# url = "https://wowpedia.fandom.com/wiki/Ragnaros"
# url = "https://wowpedia.fandom.com/wiki/Deathwing"
# url = 'https://wowpedia.fandom.com/wiki/Velen'
# url = 'https://wowpedia.fandom.com/wiki/Akama'
# url = 'https://wowpedia.fandom.com/wiki/Thargas_Anvilmar'
# url = 'https://wowpedia.fandom.com/wiki/Garrosh_Hellscream'
# url = 'https://wowpedia.fandom.com/wiki/Leoroxx'
# url = 'https://wowpedia.fandom.com/wiki/Dagran_Thaurissan'
# url = "https://wowpedia.fandom.com/wiki/C'Thun"
# url = 'https://wowpedia.fandom.com/wiki/Galakrond'


races = []
affiliations = []

for name in tqdm(list(df['Name'])):
    try:
#         name = name.split("/")[0].split("_(")[0]
        name = name.split("/")[0]
        url = "https://wowpedia.fandom.com/wiki/{}".format(name)
#         url = "https://wowpedia.fandom.com/wiki/Data_Anomaly_(Balnazzar)"
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")

        table =  soup.find("table",{"class":"infobox darktable"})

        text = table.text

        race, affiliation = get_race_and_affiliation(text)

#         print(race)
#         print(affiliation)
        

        races.append(race)
        affiliations.append(affiliation)
    except:
        print(name)
        races.append('0')
        affiliations.append('Other')
        
#     break

100%|██████████████████████████████████████████████████████████████████████████████| 2234/2234 [21:20<00:00,  1.74it/s]


In [6]:
df.loc[df['Name'] == "Sylvanas_Windrunner"]

,Name,Race,Affiliation
63,Sylvanas_Windrunner,Darkfallen,Other


In [ ]:
df.insert(1, 'Race', races)
df.insert(2, 'Affiliation', affiliations)

In [16]:
# remove the "Warcraft_Lore" and "The_Characters_of_Warcraft" characters

df = df.iloc[[i for i in range(len(df)) if 'Warcraft' not in df.iloc[i]['Name']]]
# reset index
df.reset_index(drop=True, inplace=True)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,Other
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2229,the_Jade_Warlord,Mogu,Other
2230,the_Jailer,Eternal One,Other
2231,the_Monkey_King,Hozen,Other
2232,the_Stonewright,Venthyr,Other


In [17]:
# remove stuff manually beacuse I am bad a regex

races2 = []
for race in df['Race']:
    if race != 0:
        race = race.replace("Affiliation", "").replace("Race", "").replace("Occupation", "").replace("Reaction", "").replace("Resource", "").replace("Race", "").replace("Class", "").replace("Former affiliation", "").replace("Location","").replace("Level", "").replace("Status", "")

    races2.append(race)

df['Race'] = races2

In [23]:
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)